# Model Training and Evaluation

## Imports

In [265]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score
from scipy import optimize

SEED = 333

## Data Preparation

In [266]:
df = pd.read_csv('../dataset/team_A_dataset.csv')

seasonal_cols = ['avg_monthly_salary', 'general_thefts', 'break_in_thefts', 'noveHlaseniUchazeci',
                  'absolventiSkolAMladistvi', 'noveHlasenaAUvolnenaVPM', 'obsazenaAZrusenaVPM']

for col in seasonal_cols:
    df[col + '_prev_year'] = df[col].shift(168)

#fill previous year columns for 2009 with 2009 year values
for i in range(len(df)):
    for col in seasonal_cols:
        if np.isnan(df.loc[df.index[i], col + '_prev_year']):
            df.loc[df.index[i], col + '_prev_year'] = df.loc[df.index[i], col]

encoder = OneHotEncoder(handle_unknown="ignore", drop="first")

obj_cols = df.select_dtypes('object')
encoder.fit(obj_cols)
transformed_cols = encoder.transform(obj_cols).toarray()
feature_names = encoder.get_feature_names_out()
transformed_df = pd.DataFrame(
    transformed_cols, index=df.index, columns=feature_names).astype(bool)
df = pd.concat(
    [df.select_dtypes(exclude='object'), transformed_df], axis=1)

obj_cols = df.select_dtypes('bool').columns
df[obj_cols] = df[obj_cols].astype(int)

df = df.fillna(0)

drop_cols = [f"celkem_w{w}" for w in range(2,20)]
drop_cols += [f"m_do_65_w{w}" for w in range(2,19)]
drop_cols += [f"z_do_65_w{w}" for w in range(2,19)]
drop_cols += [f"m_do_65_w{w}_ratio" for w in range(2,19)]
drop_cols += [f"z_do_65_w{w}_ratio" for w in range(2,19)]

df = df.drop(columns=drop_cols)

#Since March 2022
war_df = df.iloc[(158*14):, :]

results = []

## Model Training and Evaluation

### Evaluation Function 

In [267]:
def eval_tscv(tscv: TimeSeriesSplit, alpha: float, X, y, weights, verbose: bool = True):
    rmses = []
    maes = []
    for train_index, test_index in tscv.split(X):
        scaler = StandardScaler()
        if isinstance(X, pd.DataFrame):
            X_train = scaler.fit_transform(X.iloc[train_index, :])
            X_test = scaler.transform(X.iloc[test_index, :])
        else:
            X_train = scaler.fit_transform(X[train_index, :])
            X_test = scaler.transform(X[test_index, :])
        y_train = y[train_index]
        y_test = y[test_index]
        model = Lasso(alpha=alpha, random_state=SEED)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        mae = mean_absolute_error(y_test, y_pred)
        if verbose:
            print("pred", model.predict(X_test),
                  "test:", y_test, "rmse", rmse, "mae", mae)
            print(model.coef_)
            print(model.intercept_)
            print()
        rmses.append(rmse)
        maes.append(mae)

    weighted_rmse = np.average(rmses, weights=weights)
    weighted_mae = np.average(maes, weights=weights)
    sum_weighted_rmse = np.sum(rmses * weights)
    sum_weighted_mae = np.sum(maes * weights)
    print("rmse", weighted_rmse)
    print("mae", weighted_mae)
    print("sum weighted rmse", sum_weighted_rmse)
    print("sum weighted mae", sum_weighted_mae)

    return [weighted_rmse, weighted_mae, sum_weighted_rmse, sum_weighted_mae, model.coef_]

### Since start of the war refugees predictors present

In [268]:
drop_cols = ["celkem", "m_do_65", "z_do_65","m_do_65_ratio", "z_do_65_ratio", "uchazeciOZamestnaniUoZ",	"uchazeciOZamestnaniUoZZeny_ratio", "uchazeciOZamestnaniUoZMuzi", "uchazeciOZamestnaniUoZMuzi_ratio"]
df_war_refugees = war_df.drop(columns=drop_cols)

In [269]:
n_kraje = 14
n_splits = int(len(df_war_refugees)/n_kraje - 1)

X = df_war_refugees.loc[:, df_war_refugees.columns != 'uchazeciOZamestnaniUoZZeny'].to_numpy()
y = df_war_refugees.loc[:, df_war_refugees.columns == 'uchazeciOZamestnaniUoZZeny'].to_numpy()

tscv = TimeSeriesSplit(n_splits=n_splits, test_size=n_kraje)

In [270]:
weights = np.ones((18,))

def optimize_alpha(alpha):
    rmses = []
    for train_index, test_index in tscv.split(X):
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X[train_index, :])
        X_test = scaler.transform(X[test_index, :])
        y_train = y[train_index]
        y_test = y[test_index]

        model = Lasso(alpha=alpha, random_state=SEED, max_iter=100000)
        model.fit(X_train, y_train)

        rmse = mean_squared_error(y_test, model.predict(X_test), squared=False)
        rmses.append(rmse)

    return np.average(rmses, weights=weights)


res = optimize.minimize_scalar(
    optimize_alpha, bounds=(0, 1000), options={"disp": True})
best_alpha = res.x
print(f"Best alpha {best_alpha}")


Optimization terminated successfully;
The returned value satisfies the termination criteria
(using xtol =  1e-05 )
Best alpha 1.2296425808728362


In [271]:
q = 0.95
exps = np.linspace(0, n_splits-1, num=n_splits)
weights = np.flip(np.power(q, exps))

results.append(eval_tscv(tscv, best_alpha, X, y, weights, verbose=False))

rmse 484.2213523318167
mae 385.039490933605
sum weighted rmse 5837.633957648342
sum weighted mae 4641.9258392582715


/home/david/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.154e+04, tolerance: 4.033e+04
  model = cd_fast.enet_coordinate_descent(
/home/david/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.459e+05, tolerance: 8.020e+04
  model = cd_fast.enet_coordinate_descent(
/home/david/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.455e+

### Since start of the war no refugee predictors present

In [272]:
drop_cols = ["m_do_65_w19"]
drop_cols += ["z_do_65_w19"]
drop_cols += ["m_do_65_w19_ratio"]
drop_cols += ["z_do_65_w19_ratio"]
drop_cols += ["celkem", "m_do_65", "z_do_65","m_do_65_ratio", "z_do_65_ratio", "uchazeciOZamestnaniUoZ", "uchazeciOZamestnaniUoZZeny_ratio", "uchazeciOZamestnaniUoZMuzi", "uchazeciOZamestnaniUoZMuzi_ratio"]
df_war_no_refugees = war_df.drop(columns=drop_cols)

In [273]:
n_kraje = 14
n_splits = int(len(war_df)/n_kraje - 1)

X = df_war_no_refugees.loc[:, df_war_no_refugees.columns != 'uchazeciOZamestnaniUoZZeny'].to_numpy()
y = df_war_no_refugees.loc[:, df_war_no_refugees.columns == 'uchazeciOZamestnaniUoZZeny'].to_numpy()

tscv = TimeSeriesSplit(n_splits=n_splits, test_size=n_kraje)

In [274]:
weights = np.ones((18,))

def optimize_alpha(alpha):
    rmses = []
    for train_index, test_index in tscv.split(X):
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X[train_index, :])
        X_test = scaler.transform(X[test_index, :])
        y_train = y[train_index]
        y_test = y[test_index]

        model = Lasso(alpha=alpha, random_state=SEED, max_iter=1000000)
        model.fit(X_train, y_train)

        rmse = mean_squared_error(y_test, model.predict(X_test), squared=False)
        rmses.append(rmse)

    return np.average(rmses, weights=weights)


res = optimize.minimize_scalar(
    optimize_alpha, bounds=(0, 1000), options={"disp": True})
best_alpha = res.x
print(f"Best alpha {best_alpha}")


Optimization terminated successfully;
The returned value satisfies the termination criteria
(using xtol =  1e-05 )
Best alpha 2.5822215535124333


In [275]:
q = 0.95
exps = np.linspace(0, n_splits-1, num=n_splits)
weights = np.flip(np.power(q, exps))

results.append(eval_tscv(tscv, best_alpha, X, y, weights, verbose=False))

rmse 617.4287521848557
mae 503.4913659396094
sum weighted rmse 7443.5442237847965
sum weighted mae 6069.947723366196


/home/david/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.198e+05, tolerance: 4.033e+04
  model = cd_fast.enet_coordinate_descent(
/home/david/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.817e+05, tolerance: 8.020e+04
  model = cd_fast.enet_coordinate_descent(
/home/david/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.667e+

### Entire period refugee predictors present

In [276]:
drop_cols = ["celkem", "m_do_65", "z_do_65","m_do_65_ratio", "z_do_65_ratio", "uchazeciOZamestnaniUoZ",	"uchazeciOZamestnaniUoZZeny_ratio", "uchazeciOZamestnaniUoZMuzi", "uchazeciOZamestnaniUoZMuzi_ratio"]
df_entire_refugees = df.drop(columns=drop_cols)

In [277]:
n_kraje = 14
n_splits = int(len(war_df)/n_kraje - 1)

X = df_entire_refugees.loc[:, df_entire_refugees.columns != 'uchazeciOZamestnaniUoZZeny'].to_numpy()
y = df_entire_refugees.loc[:, df_entire_refugees.columns == 'uchazeciOZamestnaniUoZZeny'].to_numpy()

tscv = TimeSeriesSplit(n_splits=n_splits, test_size=n_kraje)

In [278]:
weights = np.ones((18,))

def optimize_alpha(alpha):
    rmses = []
    for train_index, test_index in tscv.split(X):
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X[train_index, :])
        X_test = scaler.transform(X[test_index, :])
        y_train = y[train_index]
        y_test = y[test_index]

        model = Lasso(alpha=alpha, random_state=SEED, max_iter=1000000)
        model.fit(X_train, y_train)

        rmse = mean_squared_error(y_test, model.predict(X_test), squared=False)
        rmses.append(rmse)

    return np.average(rmses, weights=weights)


res = optimize.minimize_scalar(
    optimize_alpha, bounds=(0, 1000), options={"disp": True})
best_alpha = res.x
print(f"Best alpha {best_alpha}")


Optimization terminated successfully;
The returned value satisfies the termination criteria
(using xtol =  1e-05 )
Best alpha 298.7685511573443


In [279]:
q = 0.95
exps = np.linspace(0, n_splits-1, num=n_splits)
weights = np.flip(np.power(q, exps))

results.append(eval_tscv(tscv, best_alpha, X, y, weights, False))

rmse 1486.0419447387428
mae 1079.2178589744315
sum weighted rmse 17915.29612918034
sum weighted mae 13010.741453079292


### Entire period no refugee predictors present

In [280]:
drop_cols = ["m_do_65_w19"]
drop_cols += ["z_do_65_w19"]
drop_cols += ["m_do_65_w19_ratio"]
drop_cols += ["z_do_65_w19_ratio"]
drop_cols += ["celkem", "m_do_65", "z_do_65","m_do_65_ratio", "z_do_65_ratio", "uchazeciOZamestnaniUoZ",	"uchazeciOZamestnaniUoZZeny_ratio", "uchazeciOZamestnaniUoZMuzi", "uchazeciOZamestnaniUoZMuzi_ratio"]
df_entire_no_refugees = df.drop(columns=drop_cols)

In [281]:
n_kraje = 14
n_splits = int(len(war_df)/n_kraje - 1)

X = df_entire_no_refugees.loc[:, df_entire_no_refugees.columns != 'uchazeciOZamestnaniUoZZeny'].to_numpy()
y = df_entire_no_refugees.loc[:, df_entire_no_refugees.columns == 'uchazeciOZamestnaniUoZZeny'].to_numpy()

tscv = TimeSeriesSplit(n_splits=n_splits, test_size=n_kraje)

In [282]:
weights = np.ones((18,))

def optimize_alpha(alpha):
    rmses = []
    for train_index, test_index in tscv.split(X):
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X[train_index, :])
        X_test = scaler.transform(X[test_index, :])
        y_train = y[train_index]
        y_test = y[test_index]

        model = Lasso(alpha=alpha, random_state=SEED, max_iter=1000000)
        model.fit(X_train, y_train)

        rmse = mean_squared_error(y_test, model.predict(X_test), squared=False)
        rmses.append(rmse)

    return np.average(rmses, weights=weights)


res = optimize.minimize_scalar(
    optimize_alpha, bounds=(0, 1000), options={"disp": True})
best_alpha = res.x
print(f"Best alpha {best_alpha}")


Optimization terminated successfully;
The returned value satisfies the termination criteria
(using xtol =  1e-05 )
Best alpha 298.76529218494517


In [283]:
q = 0.95
exps = np.linspace(0, n_splits-1, num=n_splits)
weights = np.flip(np.power(q, exps))

results.append(eval_tscv(tscv, best_alpha, X, y, weights, verbose=False))

rmse 1486.0419344342226
mae 1079.219964426055
sum weighted rmse 17915.296004951997
sum weighted mae 13010.766835801129


## Summary

In [293]:
model_labels = ["Since start of the war with refugees",
                "Since start of the war no refugees",
                "Entire period with refugees", 
                "Entire period no refugees"]

dfs = [df_war_refugees.drop(columns="uchazeciOZamestnaniUoZZeny"),
       df_war_no_refugees.drop(columns="uchazeciOZamestnaniUoZZeny"),
       df_entire_refugees.drop(columns="uchazeciOZamestnaniUoZZeny"),
       df_war_no_refugees.drop(columns="uchazeciOZamestnaniUoZZeny")]

feature_sel_eps = 1e-6

for i in range(len(model_labels)):
    print(model_labels[i] + ":")
    weighted_rmse, weighted_mae, sum_weighted_rmse, sum_weighted_mae, coefs = results[i]
    print("rmse", weighted_rmse)
    print("mae", weighted_mae)
    print("sum weighted rmse", sum_weighted_rmse)
    print("sum weighted mae", sum_weighted_mae)
    print("support size", np.sum((np.abs(np.array(coefs)) > feature_sel_eps).astype(int)))
    print("useful features", dfs[i].columns[np.abs(np.array(coefs)) > feature_sel_eps].to_list())
    print()

Since start of the war with refugees:
rmse 484.2213523318167
mae 385.039490933605
sum weighted rmse 5837.633957648342
sum weighted mae 4641.9258392582715
support size 38
useful features ['month', 'year', 'break_in_thefts', 'avg_monthly_salary', 'm_do_65_w19', 'z_do_65_w19', 'monthly_min_wage', 'monthly_inflation_rate_wrt_last_year', 'reer', 'm_do_65_w19_ratio', 'z_do_65_w19_ratio', 'bilance', 'avg_energy_price', 'avg_gasoline_price', 'avg_natural_gas_price', 'noveHlaseniUchazeci', 'noveHlasenaAUvolnenaVPM', 'obsazenaAZrusenaVPM', 'absolventiSkolAMladistvi', 'avg_monthly_salary_prev_year', 'general_thefts_prev_year', 'break_in_thefts_prev_year', 'noveHlaseniUchazeci_prev_year', 'absolventiSkolAMladistvi_prev_year', 'obsazenaAZrusenaVPM_prev_year', 'kraj_JHC', 'kraj_JHM', 'kraj_KVK', 'kraj_LBK', 'kraj_MSK', 'kraj_OLK', 'kraj_PAK', 'kraj_PHA', 'kraj_PLK', 'kraj_STC', 'kraj_ULK', 'kraj_VYS', 'kraj_ZLK']

Since start of the war no refugees:
rmse 617.4287521848557
mae 503.4913659396094
sum w